In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext
sc=SparkContext(master="local[3]")
print(sc)
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("spark_airport").getOrCreate()

<SparkContext master=local[3] appName=pyspark-shell>


In [31]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressionModel, GBTRegressor

from pyspark.ml.feature import VectorAssembler


In [32]:
midata = spark.read.csv('./train.csv', header = 'true', inferSchema = 'true', sep = ','  )   

In [33]:
midata.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

Hacemos un cast de nuestras variables a Double

In [34]:
import pyspark.ml.feature as ft
import pyspark.sql.types as typ
midata = midata.withColumn('YearBuilt', midata['YearBuilt'].cast(typ.DoubleType()))
midata = midata.withColumn('SalePrice', midata['SalePrice'].cast(typ.DoubleType()))
midata = midata.withColumn('TotalBsmtSF', midata['TotalBsmtSF'].cast(typ.DoubleType()))
midata = midata.withColumn('OverallQual', midata['OverallQual'].cast(typ.DoubleType()))
midata = midata.withColumn('1stFlrSF', midata['1stFlrSF'].cast(typ.DoubleType()))
midata = midata.withColumn('GrLivArea', midata['GrLivArea'].cast(typ.DoubleType()))
midata = midata.withColumn('GarageArea', midata['GarageArea'].cast(typ.DoubleType()))

In [35]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer  

In [36]:
featuresCreator = VectorAssembler(inputCols=["YearBuilt","TotalBsmtSF","OverallQual","1stFlrSF","GrLivArea","GarageArea", ],outputCol="Independent Feature" )

In [39]:
train_data,test_data=midata.randomSplit([0.80,0.20])

In [136]:
gtb =GBTRegressor(labelCol='SalePrice' ,featuresCol='Independent Feature', maxIter=100  )

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:59541)
Traceback (most recent call last):
  File "C:\spark\spark-2.3.3-bin-hadoop2.7\python\lib\py4j-0.10.7-src.zip\py4j\java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\spark\spark-2.3.3-bin-hadoop2.7\python\lib\py4j-0.10.7-src.zip\py4j\java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:59541)

In [ ]:
pipeline = Pipeline(stages=[ featuresCreator, gtb])

In [ ]:
model = pipeline.fit(train_data)

In [100]:
evaluator = RegressionEvaluator(labelCol='SalePrice', predictionCol='prediction',metricName = 'rmse')

Error cuadratico medio mide el promedio de los errores al cuadrado, es decir, la diferencia entre el estimador y lo que se estima

In [101]:
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
print("MSE: %.3f" % rmse)

MSE: 24268.008


El coeficiente de determinacion determina la calidad del modelo para Predecir los resultados

In [102]:
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
print("r2: %.3f" %r2)

r2: 0.907


In [115]:
import pyspark.ml.tuning as tune

In [ ]:
gts =GBTRegressor(labelCol='SalePrice' ,featuresCol='Independent Feature' )

In [128]:
paramGrid = ParamGridBuilder().build()
grid = tune.ParamGridBuilder().addGrid(gts.maxIter, [10,30]).addGrid(gts.maxDepth, [5,10])

In [129]:
crossval = CrossValidator(estimator=pipeline,
 estimatorParamMaps=paramGrid,
 evaluator=evaluator, 
 numFolds=5)

In [130]:
CV_model = crossval.fit(train_data,)
predictions_improved = CV_model.bestModel.transform(test_data,)

In [131]:
results = [
    (
        [
            {key.name: paramValue} 
            for key, paramValue 
            in zip(
                params.keys(), 
                params.values())
        ], metric
    ) 
    for params, metric 
    in zip(
        CV_model.getEstimatorParamMaps(), 
        CV_model.avgMetrics
    )
]

sorted(results, key=lambda el: el[1], reverse=True)[0]

([], 40493.97193136773)

In [126]:
results = [
    (
        [
            {key.name: paramValue} 
            for key, paramValue 
            in zip(
                params.keys(), 
                params.values())
        ], metric
    ) 
    for params, metric 
    in zip(
        #cvModel.getEstimatorParamMaps(), 
        #cvModel.avgMetrics
        CV_model.getParam('maxIter')
        
    )
]

sorted(results, key=lambda el: el[1], reverse=True)[0]

AttributeError: 'CrossValidatorModel' object has no attribute 'maxIter'